<a href="https://colab.research.google.com/github/NikhithaKunati/INFO5731_Spring2020/blob/master/Copy_of_INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


**PART - 1**

In [79]:
# Write your code here
import nltk
nltk.download('punkt')
from tabulate import tabulate

f = open('/content/clean.txt','r')
raw = f.read()
res = []

#to get tokens
tokens = nltk.word_tokenize(raw)

#Create your trigrams
bgs = nltk.trigrams(tokens)

#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(bgs)
for k,v in fdist.items():
  res.append([k,v])

print(tabulate(res))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
---------------------------------------------------------------------  -
('solomonyue', 'china', 'ccp')                                         3
('china', 'ccp', 'operative')                                          3
('ccp', 'operative', 'exposed')                                        3
('operative', 'exposed', 'ejected')                                    3
('exposed', 'ejected', 'thanks')                                       3
('ejected', 'thanks', 'wuhancoronavirus')                              3
('thanks', 'wuhancoronavirus', 'amp')                                  3
('wuhancoronavirus', 'amp', 'china')                                   3
('amp', 'china', 'blaming')                                            3
('china', 'blaming', 'victim')                                         3
('blaming', 'victim', 'u')                                             3
('victim', 'u

**PART - 2**

In [80]:
def bi_gram_prob(file):
    uni_grams = {} #to store all uni_grams
    bi_grams = {} #to store all bi_grams
    f = open(file, 'r')
    text = f.read()

    for i in tokens:
        if not i in uni_grams:
            uni_grams[i] = 1
        else:
            uni_grams[i] += 1

    for i in range(len(tokens) - 1):
        temp = (tokens[i], tokens[i+1])
        if not temp in bi_grams:
            bi_grams[temp] = 1
        else:
            bi_grams[temp] += 1
    
    res = []
    for k,v in bi_grams.items():
        f_word = k[0]
        bi_prob = bi_grams[k] / uni_grams[f_word]
        res.append([*k,bi_prob])
        print(tabulate(res))

bi_gram_prob('clean.txt') #to call the defined function

Streaming output truncated to the last 5000 lines.
assistance                   worker                       1
worker                       amp                          1
amp                          business                     0.166667
business                     senschumer                   1
senschumer                   im                           1
im                           brooklyn                     1
brooklyn                     amp                          1
amp                          speakerpelosixexxa           0.166667
speakerpelosixexxa           bwuhanvirus                  1
bwuhanvirus                  wuhancoronavirus             1
wuhancoronavirus             wuhanpneumonia               0.0238095
wuhanpneumonia               ccpvirus                     1
ccpvirus                     covidxexxbc                  1
covidxexxbc                  wholiedpeopledied            1
wholiedpeopledied            ncan                         1
ncan                       

**PART - 3**

In [81]:
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree
import pandas as pd

data = []
with open("clean.txt", "r") as fp:
    data = fp.readlines()

df = pd.DataFrame({'text':data})
res = []
def get_chunks(text, chunk_func=ne_chunk):
    parts = chunk_func(pos_tag(word_tokenize(text)))
    chunk = []
    present = []

    for a in parts:
        if type(a) == Tree:
            present.append(" ".join([token for token, pos in a.leaves()]))
        elif present:
            a = " ".join(present)
            if a not in chunk:
                chunk.append(a)
        else:
            continue
    res.append(chunk)
    return chunk

df['text'].apply(lambda sent: get_chunks((sent)))
lists = [item for sublist in res for item in sublist]
print(lists)

['TIFF', 'TIFF MASTERPIECE', 'TIFF MASTERPIECE Phoenix']


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

**PART - 1**

In [82]:
# Write your code here
import numpy 
import bs4 as bs
import heapq
from tabulate import tabulate
from scipy import spatial

#read the required file
f = open('/content/clean.txt','r')
text = f.read()

#to store all required values
freq = {}
idf_values = {}
tf_values = {}
tf_idf = []
copy_text = nltk.sent_tokenize(text) # stores a sentence tokenized copy
for a in copy_text:
    tokens = nltk.word_tokenize(a) 
    for i in tokens:
        if i not in freq.keys():
            freq[i] = 1
        else:
            freq[i] += 1
all_freq = heapq.nlargest(200, freq, key=freq.get)

for i in all_freq:
    count = 0
    for document in copy_text:
        if i in nltk.word_tokenize(document):
            count += 1
    idf_values[i] = numpy.log(len(copy_text)/(1 + count))

for i in all_freq:
    tf = []
    for document in copy_text:
        doc_freq = 0
        for word in nltk.word_tokenize(document):
            if i == word:
                  doc_freq += 1
        word_tf = doc_freq/len(nltk.word_tokenize(document))
        tf.append(word_tf)
    tf_values[i] = tf

for i in tf_values.keys():
    tfidf_sentences = []
    for k in tf_values[i]:
        tf_idfs = k * idf_values[i]
        tf_idfs = float(round(tf_idfs, 3))
        tfidf_sentences.append(tf_idfs)
    tf_idf.append([i] + tfidf_sentences)
print(tabulate(tf_idf,headers=("token","tf_idf value")))

token                          tf_idf value
---------------------------  --------------
wuhancoronavirus                     -0.026
realjameswoods                       -0.023
china                                -0.008
university                           -0.007
solomonyue                           -0.006
international                        -0.005
thexexxa                             -0.005
democrat                             -0.005
u                                    -0.004
nancypelosi                          -0.004
harvard                              -0.004
wuhan                                -0.004
pandemic                             -0.004
amp                                  -0.004
uh                                   -0.004
fact                                 -0.004
chinaambindia                        -0.004
chinese                              -0.004
thanks                               -0.003
txexxa                               -0.003
lose                            

**PART - 2**

In [83]:
import nltk
a = ['']
cosines = [a]
for d1 in tf_idf:
    data1 = d1[1:]
    cosine  = [d1[0]]
    a.append(d1[0])
    for d2 in tf_idf:
       data2 = d2[1:]
       result = 1 - spatial.distance.cosine(data1, data2)
       cosine.append(result)
    cosines.append(cosine)
print(tabulate(cosines))

---------------------------  ------------------  ------------------  ------------------  ------------------  ----------  -------------  --------  --------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------  ------  ----  -----  ---  ------  -------------  ----  ---------  ---------  -----------  -------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  ------------------  -----------------

# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: https://github.com/NikhithaKunati/INFO5731_Spring2020/blob/master/WuhanCoronaVirus%20Tweet%20Reaction.csv